In [ ]:
import pandas  as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import roc_auc_score
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
train = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
train["missing"] = train.isnull().sum(axis = 1)
test["missing"] = test.isnull().sum(axis = 1)

In [ ]:
target = "claim"
predictors = [x for x in train.columns if x not in ["id", target]]

kf = KFold(n_splits = 5, shuffle = True, random_state = 666)
skf = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 666)

In [ ]:
train[predictors] = train[predictors].fillna(train.groupby("missing")[predictors].transform("mean"))
test[predictors] = test[predictors].fillna(train.groupby("missing")[predictors].transform("mean"))

In [ ]:
scaler = StandardScaler()

train[predictors] = scaler.fit_transform(train[predictors])
test[predictors] = scaler.transform(test[predictors])

In [ ]:
X = train[predictors]
y = train[target]
test = test[predictors]

In [ ]:
params = {
    'max_depth': 6, 
    'colsample_bytree': 0.7098433872257219, 
    'min_child_weight': 482, 
    'subsample': 0.8406820875269025, 
    'reg_alpha': 3.2594867475105374, 
    'reg_lambda': 0.1534227221930378
}

In [ ]:
oof_xgb = np.zeros(len(X))
predictions_xgb = np.zeros(len(test))
i = 1

for train_ix, test_ix in kf.split(X.values):
        
    print("\033[1m" + "Out of fold predictions generating for fold \033[94m {} \033[0m \n".format(i))
        
    train_X, train_y = X.values[train_ix], y.values[train_ix]
    test_X, test_y = X.values[test_ix], y.values[test_ix]
    
    model_xgb = xgb.XGBClassifier(
        random_state = 666,
        n_jobs = -1,
        n_estimators = 40000,
        learning_rate = 0.005,
        tree_method = "gpu_hist",
        predictor = "gpu_predictor",
        gpu_id = 0,
        **params
    )   
    
    model_xgb.fit(
        train_X, 
        train_y,
        eval_set = [(test_X, test_y)],
        eval_metric = "auc",
        early_stopping_rounds = 300,
        verbose = 1000,
    )
    
    oof_xgb[test_ix] = oof_xgb[test_ix] + model_xgb.predict_proba(test_X)[:, 1]
    predictions_xgb = predictions_xgb + model_xgb.predict_proba(test)[:, 1]
    
    print("\033[1mAUC for fold \033[91m{} \t\t\t \033[92m {} \033[0m \n".format(i, round(roc_auc_score(test_y, oof_xgb[test_ix]), 5)))
    
    i = i + 1
    
print("\033[1mAUC for Training Set: \t\t \033[92m {} \033[0m \n".format(round(roc_auc_score(y, oof_xgb), 5)))

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-sep-2021/sample_solution.csv")
submission[target] = predictions_xgb / 5
submission.to_csv("submission.csv", index = False)
submission

In [ ]:
np.save("oof_xgb.npy", oof_xgb)
np.save("predictions_xgb.npy", predictions_xgb)